# Practica 1B
##  Arturo Ibáñez Martínez
## Jorge del Valle Vázquez

In [63]:
cd aima-python

[WinError 2] El sistema no puede encontrar el archivo especificado: 'aima-python'
C:\Users\uSUARIO\Ejercicios IA\Practica1b


In [2]:
# Cargamos el módulo con los algoritmos de búsqueda.
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

In [3]:
class Problem(object):

    def __init__(self, initial, goal=None):
        
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        raise NotImplementedError

    def result(self, state, action):
        raise NotImplementedError

    def goal_test(self, state):
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal
        
    def path_cost(self, c, state1, action, state2):
        return c + 1

    def value(self, state):
        raise NotImplementedError

    def coste_de_aplicar_accion(self, estado, accion):
        return 1

In [4]:
class Problema_con_Analizados(Problem):
         
    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados  = 0
        self.goal = problem.goal

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado,accion)

In [30]:
def resuelve_analizando(pca, algoritmo, h=None):

    if h: 
        sol= algoritmo(pca,h).solution()
    else: 
        sol= algoritmo(pca).solution()
    print("Solución: {0}".format(sol))
    print("Algoritmo: {0}".format(algoritmo.__name__))
    if h: 
        print("Heurística: {0}".format(h.__name__))
    else:
        pass
    print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),pca.analizados))

## Problema Puente
### Comentarios
#### - Utilizamos tanto bfts y dfts. Para el problema planteado la busqueda en anchura analiza menos nodos, es decir, encuentra antes la solucion.
#### - No podemos usar graph search puesto que no se puede hacer hash de lista. Igual con a*. Podríamos haber representado los conjuntos con binarios que podemos expresar con enteros y ya podríamos hacer hash.
#### - Para representar el problema cada estado almacena la gente que queda por cruzar , en que lado del puente se encuentra la linterna y el tiempo de uso de la literna acumulado. El estado objetivo es aquel en el que han cruzado todos la linterna queda a la derecha y les ha durado la linterna encendida hasta el final o sobra algo de batería 
#### - La lista cruzar almacena los tiempos que tarda cada persona en cruzar
#### - Las acciones vienen dadas por la persona o la pareja que procede a cruzar el puente
#### - Heuristica : consideramos la heurística dada por 2*C +linterna -1 , donde C es la gente que falsta por cruzar, linterna vale 0 en el origen y 1 en el destino, el -1 para tener valor 0 en el estado objetivo

In [36]:
# Creamos la clase ProblemaPuente con los elementos que representarán el problema. 
class Puente(Problem):
    def __init__(self, initial, cruzar, goal):
        '''Inicializacion de nuestro problema.'''
        Problem.__init__(self, initial, goal)
        self.cruzar= cruzar
        self._actions = ['1','2', '3','4','5','12','23','34','45','13','24','35','14','25','15']

    def goal_test(self, state):
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state[0] == self.goal[0] and state[1] == self.goal[1] and state[2] <= self.goal[2]
    def actions(self, s):
        '''Devuelve las acciones validas para un estado.'''
        return [a for a in self._actions if self._is_valid(s,a)]

    def _is_valid(self, s,a):
        '''Determina si un estado es valido o no.'''
        t1 = self.cruzar[int(a[0])-1]
        t2= 0
        if len(a)>1:
            t2 = self.cruzar[int(a[1])-1]
        t3 = max(t1, t2)
        return (s[2]+ t3 <=self.goal[2]) and ((s[1] == 0 and (int(a[0]) in s[0]) and ((len(a)<2) or (int(a[1]) in s[0]))) or (s[1] == 1 and (not(int(a[0]) in s[0])) and (len(a)<2 or not(int(a[1]) in s[0])))) and  1 <= int(a[0]) <= 5 and (len(a)<2 or 0 <= int(a[1])<= 5) 

    def result(self, s, a):
        '''Devuelve el estado resultante de aplicar una accion a un estado
           determinado.'''
        estado = s
        conjunto=set(estado[0])
        linterna=estado[1]
        tiempo=estado[2]
        t1 = self.cruzar[int(a[0])-1]
        t2= 0
        if len(a)>1:
            t2 = self.cruzar[int(a[1])-1]
        t3 = max(t1, t2)
        if linterna == 0:
            conjunto.discard(int(a[0]))
            if len(a)>1:
                conjunto.discard(int(a[1]))
            return (list(conjunto), 1, tiempo+t3)
        else:
            conjunto.add(int(a[0]))
            if len(a)>1:
                conjunto.add(int(a[1]))
            return (list(conjunto), 0,tiempo+t3)
        
    def h(self, node):
        """ Return the heuristic value for a given state. """
        state= node.state
        c = len(state[0])
        l= state[1]
        return (2*c) + l - 1

In [38]:
p = Puente(([1,2,3,4,5], 0, 0), [10,30,60,80,120],(list(), 1, 300))

In [39]:
depth_first_tree_search(p).solution()

['13', '1', '12', '2', '45', '1', '12']

In [40]:
breadth_first_tree_search(p).solution()

['12', '1', '45', '2', '12', '1', '13']

In [62]:
# Aparece un error si incluyo la heuristica
resuelve_analizando(Problema_con_Analizados(p),depth_first_tree_search)

Solución: ['13', '1', '12', '2', '45', '1', '12']
Algoritmo: depth_first_tree_search
Longitud de la solución: 7. Nodos analizados: 26338


In [47]:
resuelve_analizando(Problema_con_Analizados(p),breadth_first_tree_search)

Solución: ['12', '1', '45', '2', '12', '1', '13']
Algoritmo: breadth_first_tree_search
Longitud de la solución: 7. Nodos analizados: 15859


## Problema Puzzle matemático
### Comentarios
#### -  Usamos para resolverlo dfs (tree o graph) o bfs (tree). Para este problema ambas son completas; pero dfs es mucho más eficiente. Esto se debe a que dfs encuentra rápido la primera solucion y bfs analiza todos los nodados hasta llegar al ultimo nivel.
#### -  Tanto a* como bfgs no llegan a acabar. Desconocemos por qué.
#### -  Para representarlo hemos considerado 25 casillas, donde 12 son de operaciones y 13 de posiciones de operandos, de los que 4 vienen precolocados.
#### - La lista "operaciones" bien podría ser una entrada del problema en la que se nos especifica cuál es el tablero a relleanar, siendo "Comienzo" la primera casilla en caso de estar vacía; los nombres de operaciones, una casilla con dicha operación seguida de una casilla vacía, y los nombres de operaciones seguidos de una cifra, dos casillas consecutivas no vacías.En el caso resuelto, el tamaño de la lista es 13.
#### -El algoritmo resuelve el problema para un caso general (modificando los estados iniciales y final, asi como la lista operaciones)
#### - En el estado se guardan una tupla con las cifras todavía no colocadas,el índice que marca la casilla actual para conocer la operación cerrespondiente y el valor acumulado. El estado objetivo es aquel donde no quedan  cifras por colocar, se han ocupado todas las casillas y se tiene el valor esperado. Para el mismo problema en un caso general, el estado onjetivo sería la tupla vacía, el tamaño de la lista operaciones, y el resultado que debe dar la sucesión de operaciones.
#### - El array operaciones concreta cómo es el tablero, es decir, la sucesión de operaciones a realizar.
#### - No consideramos la posibilidad de obtener el resultado con algun casilla sin colocar.
#### - Las acciones consideradas son o bien operar una casilla precolocada o bien colocar una cifra y realizar la operación correspondiente. 
#### - Coste: No hemos redefinido la función coste_de_aplicar_accion ya que ninguna acción posee un coste distinto.
#### - Heuristica: No se puede definir heurística alguna puesto que es imposible determinar si un estado tiene más posibilidades de alcanzar una solución que otro, puesto que solamente realizando todas las operaciones sabremos si el resultado final es el deseado o no.

In [49]:
operaciones = ["Comienzo","Suma 13","Multiplicacion","Division","Suma","Suma 12","Multiplicacion","Resta","Resta 11","Suma","Multiplicacion","Division","Resta 10"]
class Puzzle(Problem):
    """estado : numeros por colocar, casillas con las que hemos operado(incluyendo las prefijadas), valor actual"""
    def __init__ (self,initial = ((1,2,3,4,5,6,7,8,9),0,0),goal = ((),13,66)):
        self.initial= initial
        self.goal = goal
        
    def actions (self,estado):
        indice = estado[1]
        if indice < len(operaciones):
            operacion = operaciones[indice].split()
            if len(operacion) == 2 :
                return [operaciones[indice]]
            else:
                accs = list()
                for i in range(0,len(estado[0])):
                    accs.append("Colocar el numero " + str(estado[0][i]))
                return accs
        else  : return []
        
    def result (self,estado,accion):
        l = list(estado[0])
        indice = estado[1]
        num = estado[2]
        operacion = accion.split()
        if len(operacion) == 2 :
            if operacion[0] == "Suma":
                            num += int(operacion[1])
            elif operacion[0] == "Multiplicacion":
                            num *= int(operacion[1])
            elif operacion[0] == "Division":
                            num /= int(operacion[1])
            elif operacion[0] == "Resta":
                            num -= int(operacion[1])
        else:
            for elem in l:
                if accion == "Colocar el numero " + str(elem):
                        l.remove(elem)
                        if operaciones[indice] == "Comienzo":
                            num = elem
                        elif operaciones[indice] == "Suma":
                            num += elem
                        elif operaciones[indice] == "Multiplicacion":
                            num *= elem
                        elif operaciones[indice] == "Division":
                            num /= elem
                        elif operaciones[indice] == "Resta":
                            num -= elem
                            
        return (tuple(l),indice+1,num)
    
    def h(self, node):
        """ Return the heuristic value for a given state. """
        return 1

In [50]:
breadth_first_tree_search(Puzzle()).solution()

['Colocar el numero 1',
 'Suma 13',
 'Colocar el numero 2',
 'Colocar el numero 4',
 'Colocar el numero 7',
 'Suma 12',
 'Colocar el numero 5',
 'Colocar el numero 8',
 'Resta 11',
 'Colocar el numero 3',
 'Colocar el numero 6',
 'Colocar el numero 9',
 'Resta 10']

In [52]:
depth_first_graph_search(Puzzle()).solution()

['Colocar el numero 9',
 'Suma 13',
 'Colocar el numero 7',
 'Colocar el numero 2',
 'Colocar el numero 8',
 'Suma 12',
 'Colocar el numero 4',
 'Colocar el numero 3',
 'Resta 11',
 'Colocar el numero 6',
 'Colocar el numero 1',
 'Colocar el numero 5',
 'Resta 10']

In [53]:
depth_first_tree_search(Puzzle()).solution()

['Colocar el numero 9',
 'Suma 13',
 'Colocar el numero 7',
 'Colocar el numero 2',
 'Colocar el numero 8',
 'Suma 12',
 'Colocar el numero 4',
 'Colocar el numero 3',
 'Resta 11',
 'Colocar el numero 6',
 'Colocar el numero 1',
 'Colocar el numero 5',
 'Resta 10']

In [54]:
resuelve_analizando(Problema_con_Analizados(Puzzle()),breadth_first_tree_search)

Solución: ['Colocar el numero 1', 'Suma 13', 'Colocar el numero 2', 'Colocar el numero 4', 'Colocar el numero 7', 'Suma 12', 'Colocar el numero 5', 'Colocar el numero 8', 'Resta 11', 'Colocar el numero 3', 'Colocar el numero 6', 'Colocar el numero 9', 'Resta 10']
Algoritmo: breadth_first_tree_search
Longitud de la solución: 13. Nodos analizados: 1051040


In [55]:
resuelve_analizando(Problema_con_Analizados(Puzzle()),depth_first_tree_search)

Solución: ['Colocar el numero 9', 'Suma 13', 'Colocar el numero 7', 'Colocar el numero 2', 'Colocar el numero 8', 'Suma 12', 'Colocar el numero 4', 'Colocar el numero 3', 'Resta 11', 'Colocar el numero 6', 'Colocar el numero 1', 'Colocar el numero 5', 'Resta 10']
Algoritmo: depth_first_tree_search
Longitud de la solución: 13. Nodos analizados: 33886


In [56]:
resuelve_analizando(Problema_con_Analizados(Puzzle()),depth_first_graph_search)

Solución: ['Colocar el numero 9', 'Suma 13', 'Colocar el numero 7', 'Colocar el numero 2', 'Colocar el numero 8', 'Suma 12', 'Colocar el numero 4', 'Colocar el numero 3', 'Resta 11', 'Colocar el numero 6', 'Colocar el numero 1', 'Colocar el numero 5', 'Resta 10']
Algoritmo: depth_first_graph_search
Longitud de la solución: 13. Nodos analizados: 27928
